Тема: Разработка информационной системы для учета заказов компании
Цель задания закрепить основы ОоП такие как работа с классами, коллекциями, делегатами и событиями. В результате вы создадите модель, в которой будут участвовать сущности «Компания», «Сотрудник», «Заказ», и «Заказчик». Проект должен содержать функционал для обработки событий обновления данных, а так же использовать делегаты для уведомлений.
Описание задания
Вам необходимо разработать консольное приложение на языке программирования C#, которое моделирует работу компании по учету заказов. Программа должна использовать классы для описания сущностей их атрибутов и методов. Для  реализации логики будут использоваться коллекции для хранения связанных, данных, а также делегаты и события для автоматизации уведомлений о ключевых действиях в системе


In [1]:
using System;
using System.Collections.Generic;

public class Company
{
    public string Name { get; set; }
    public List<Employee> Employees { get; set; } = new List<Employee>();
    public List<Order> Orders { get; set; } = new List<Order>();

    public Company(string name)
    {
        Name = name;
    }

    public void AddEmployee(Employee employee)
    {
        Employees.Add(employee);
        Console.WriteLine($"Сотрудник {employee.FirstName} {employee.LastName} добавлен в компанию.");
    }

    public void AddOrder(Order order)
    {
        Orders.Add(order);
        Console.WriteLine($"Заказ {order.OrderId} добавлен в компанию.");
    }

    public void AssignEmployeeToOrder(int orderId, Employee employee)
    {
        var order = Orders.Find(o => o.OrderId == orderId);
        if (order != null)
        {
            order.AssignedEmployee = employee;
            employee.OrdersProcessed++;
            Console.WriteLine($"Сотрудник {employee.FirstName} {employee.LastName} назначен на заказ {orderId}.");
        }
    }

    public void ChangeOrderStatus(int orderId, OrderStatus newStatus)
    {
        var order = Orders.Find(o => o.OrderId == orderId);
        if (order != null)
        {
            order.ChangeStatus(newStatus);
        }
    }

    public List<Order> GetOrdersByStatus(OrderStatus status)
    {
        var orders = Orders.FindAll(o => o.Status == status);
        Console.WriteLine($"Заказы со статусом {status}:");
        foreach (var order in orders)
        {
            Console.WriteLine($"Заказ {order.OrderId}: {order.Description}");
        }
        return orders;
    }

    public List<Order> GetOrdersByCustomer(Customer customer)
    {
        var orders = Orders.FindAll(o => o.Customer == customer);
        Console.WriteLine($"Заказы заказчика {customer.FirstName} {customer.LastName}:");
        foreach (var order in orders)
        {
            Console.WriteLine($"Заказ {order.OrderId}: {order.Description}");
        }
        return orders;
    }

    public void GenerateEmployeeReport()
    {
        Console.WriteLine("Отчет по сотрудникам:");
        foreach (var employee in Employees)
        {
            Console.WriteLine($"Сотрудник: {employee.FirstName} {employee.LastName}, Обработано заказов: {employee.OrdersProcessed}");
        }
    }
}
public class Employee
{
    public string FirstName { get; set; }
    public string LastName { get; set; }
    public string Position { get; set; }
    public int OrdersProcessed { get; set; }

    public Employee(string firstName, string lastName, string position)
    {
        FirstName = firstName;
        LastName = lastName;
        Position = position;
    }

    public void NotifyEmployee(string message)
    {
        Console.WriteLine($"Уведомление сотрудника {FirstName} {LastName}: {message}");
    }
}
public class Customer
{
    public string FirstName { get; set; }
    public string LastName { get; set; }
    public string PhoneNumber { get; set; }

    public Customer(string firstName, string lastName, string phoneNumber)
    {
        FirstName = firstName;
        LastName = lastName;
        PhoneNumber = phoneNumber;
    }
}
public enum OrderStatus
{
    Created,
    InProgress,
    Completed
}

public class Order
{
    public int OrderId { get; }
    public string Description { get; set; }
    public Employee AssignedEmployee { get; set; }
    public OrderStatus Status { get; set; }
    public DateTime CreatedAt { get; }
    public Customer Customer { get; set; }

    public delegate void OrderStatusHandler(string message);
    public event OrderStatusHandler OrderStatusChanged;

    private static int nextOrderId = 1;

    public Order(string description, Customer customer)
    {
        OrderId = nextOrderId++;
        Description = description;
        Status = OrderStatus.Created;
        CreatedAt = DateTime.Now;
        Customer = customer;
        Console.WriteLine($"Заказ {OrderId} создан: {Description}");
    }

    public void ChangeStatus(OrderStatus newStatus)
    {
        Status = newStatus;
        OrderStatusChanged?.Invoke($"Статус заказа {OrderId} изменен на {newStatus}");
        Console.WriteLine($"Статус заказа {OrderId} изменен на {newStatus}");
    }
}

Company company = new Company("Tyuiu");

Employee employee1 = new Employee("Иван", "Иванов", "Менеджер");
Employee employee2 = new Employee("Анна", "Анновна", "Разработчик");

Customer customer1 = new Customer("Алексей", "Алексеев", "123-456-789");
Customer customer2 = new Customer("Юлия", "Юльевна", "987-654-321");

company.AddEmployee(employee1);
company.AddEmployee(employee2);

Order order1 = new Order("Разработка сайта", customer1);
Order order2 = new Order("Техническая поддержка", customer2);

company.AddOrder(order1);
company.AddOrder(order2);

company.AssignEmployeeToOrder(order1.OrderId, employee1);
company.AssignEmployeeToOrder(order2.OrderId, employee2);

order1.OrderStatusChanged += employee1.NotifyEmployee;
order2.OrderStatusChanged += employee2.NotifyEmployee;

company.ChangeOrderStatus(order1.OrderId, OrderStatus.InProgress);
company.ChangeOrderStatus(order2.OrderId, OrderStatus.Completed);

var inProgressOrders = company.GetOrdersByStatus(OrderStatus.InProgress);
var completedOrders = company.GetOrdersByStatus(OrderStatus.Completed);

var customer1Orders = company.GetOrdersByCustomer(customer1);
var customer2Orders = company.GetOrdersByCustomer(customer2);

company.GenerateEmployeeReport();

Сотрудник Иван Иванов добавлен в компанию.
Сотрудник Анна Анновна добавлен в компанию.
Заказ 1 создан: Разработка сайта
Заказ 2 создан: Техническая поддержка
Заказ 1 добавлен в компанию.
Заказ 2 добавлен в компанию.
Сотрудник Иван Иванов назначен на заказ 1.
Сотрудник Анна Анновна назначен на заказ 2.
Уведомление сотрудника Иван Иванов: Статус заказа 1 изменен на InProgress
Статус заказа 1 изменен на InProgress
Уведомление сотрудника Анна Анновна: Статус заказа 2 изменен на Completed
Статус заказа 2 изменен на Completed
Заказы со статусом InProgress:
Заказ 1: Разработка сайта
Заказы со статусом Completed:
Заказ 2: Техническая поддержка
Заказы заказчика Алексей Алексеев:
Заказ 1: Разработка сайта
Заказы заказчика Юлия Юльевна:
Заказ 2: Техническая поддержка
Отчет по сотрудникам:
Сотрудник: Иван Иванов, Обработано заказов: 1
Сотрудник: Анна Анновна, Обработано заказов: 1
